In [ ]:

kyukaiiki = """
[EbM7] [BbM7/F]

[EbM7]腐り出したこの地に
花は咲かない
ただ灰が積[BbM7/F]もっていく

[EbM7]諦めかけた想いが胸を刺す 
船を出す　
[BbM7/F]あの海へと

[EbM7] [BbM7/F]

[EbM7]何も光らない 
何も祈れない 
ただ星だ [BbM7/F]けが見える

[EbM7]夜が来るたび
もう帰れない
壊れた[BbM7/F]故郷を

[Bb/G]想う [Bb/F]思う [Bb/Eb] [Bb7]

旧海域[Bb/Eb]はまだ[Bb/F]遠く
[Bb/G]光の名前を[Bb/Ab]頼りに[Bb/Eb]夜を[Bb/F]往こう[Bb/G] [Bb/Ab]
[Bb/Eb]覚えた[Bb/F]歌を[Bb/G]口ずさみながら[Bb/Ab]
[Bb/Eb]進[Bb/F]むよ

([Bb/Eb]|----|----|----[Bb/F]|----)*4
"""

# 文章中の[]を<span class="chord"></span>に変換
kyukaiiki = kyukaiiki.replace("[", "<span class=\"chord\">").replace("]", "</span>")
print(kyukaiiki)



In [ ]:
def to_html(t):
    # 文章中の[]を<span class="chord"></span>に変換
    t = t.replace("[", "<span class=\"chord\">").replace("]", "</span>")
    # 文章中の改行を<br>に変換
    t = t.replace("\n", "<br>")
    return t